# Rock, Paper, and Scissors

## i'm running it in Google Collab


In [10]:
from os import getcwd

import tensorflow as tf
import tensorflow_datasets as tfds

## Load mobilenet model and its features


In [11]:
import tensorflow_hub as hub

model_selection = ("mobilenet_v2", 224, 1280)
handle_base, pixels, FV_SIZE = model_selection
IMAGE_SIZE = (pixels, pixels)

MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(
    handle_base
)
feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_SIZE + (3,))
feature_extractor.trainable = True

## Make one hot encoding for the labels. the labels have 3 classes


In [12]:
def OneHotEncoding(feature, label):
    one_hot = tf.one_hot(label, 3)
    return feature, one_hot


# Test the function
_, one_hot = OneHotEncoding(["a", "b", "c", "a", "b"], [1, 2, 3, 1, 2])
print(one_hot)

tf.Tensor(
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]], shape=(5, 3), dtype=float32)


## Load The Dataset and Explore it


In [13]:
filepath = f"{getcwd()}/data"

train_data = tfds.load(
    "rock_paper_scissors:3.*.*", split="train", data_dir=filepath, as_supervised=True
)
test_data = tfds.load(
    "rock_paper_scissors:3.*.*", split="test", data_dir=filepath, as_supervised=True
)

# Knowing the amount of data
print(f"Amount of train data: {len(list(train_data))}")
print(f"Amount of test data: {len(list(test_data))}")

# Knowing the shape of image
_, info = tfds.load("rock_paper_scissors:3.*.*", with_info=True, data_dir=filepath)

# DO NOT EDIT THIS
print(info.features["image"].shape)

Amount of train data: 2520
Amount of test data: 372
(300, 300, 3)


## Mapping dataset use one hot encode function


In [14]:
train_data = train_data.map(OneHotEncoding)
test_data = test_data.map(OneHotEncoding)

print(type(train_data))

<class 'tensorflow.python.data.ops.map_op._MapDataset'>


In [15]:
def resize_image(image, label):
    image = tf.image.resize(image, [224, 224])
    return image, label

train_data = train_data.map(resize_image)
test_data = test_data.map(resize_image)

print(type(train_data))

<class 'tensorflow.python.data.ops.map_op._MapDataset'>


## Use model Transfer Learning


In [16]:
model = tf.keras.Sequential(
    [
        feature_extractor,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 flatten_1 (Flatten)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 2227715 (8.50 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________


## Evaluate model


In [17]:
train_batches = train_data.shuffle(100).batch(10)
validation_batches = test_data.batch(32)

In [18]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get("accuracy") > 0.96 and logs.get("val_accuracy") > 0.96:
            print("\nReached 96% accuracy so cancelling training!")
            self.model.stop_training = True

# Buat instance dari callback
my_callback = MyCallback()

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

history = model.fit(
    train_batches, epochs=1000, validation_data=validation_batches,callbacks=[my_callback],steps_per_epoch=7
)

print("Final Training Accuracy:-", history.history["accuracy"][-1])
print("Final Validation Accuracy:-", history.history["val_accuracy"][-1])

Epoch 1/1000
7/7 [==============================] - 54s 5s/step - loss: 0.3937 - accuracy: 0.8393 - val_loss: 0.9566 - val_accuracy: 0.5780
Epoch 2/1000
7/7 [==============================] - 34s 5s/step - loss: 0.2730 - accuracy: 0.9464 - val_loss: 3.0787 - val_accuracy: 0.6667
Epoch 3/1000
7/7 [==============================] - 42s 7s/step - loss: 0.3183 - accuracy: 0.9286 - val_loss: 11.8986 - val_accuracy: 0.5860
Epoch 4/1000
7/7 [==============================] - 43s 7s/step - loss: 0.1577 - accuracy: 0.9643 - val_loss: 26.4999 - val_accuracy: 0.3333
Epoch 5/1000
7/7 [==============================] - 33s 5s/step - loss: 0.1398 - accuracy: 0.9732 - val_loss: 20.4926 - val_accuracy: 0.3548
Epoch 6/1000
7/7 [==============================] - 33s 5s/step - loss: 0.5697 - accuracy: 0.9375 - val_loss: 16.9628 - val_accuracy: 0.4758
Epoch 7/1000
7/7 [==============================] - 42s 6s/step - loss: 0.1791 - accuracy: 0.9554 - val_loss: 10.7459 - val_accuracy: 0.6586
Epoch 8/1000
7/